# la representation TFIDF


In [8]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import re
import string

In [2]:
#DATASET_FILE = "./dataset_cleaned.csv"

In [4]:
#dataset_df = pd.read_csv(DATASET_FILE)
#dataset_df

In [9]:
# expand pandas df column display width to enable easy inspection
pd.set_option('max_colwidth', 150)
# read in csv to dataframe
df = pd.read_csv('dataset_cleaned.csv', engine='python')
# visually inspect dataframe
df=df[df['stars'].isin([1,2])]
df

,text,stars,length,text_cleaned
0,"I've only had food from here once and it wasn't memorable. I had Panang curry and the balance in flavor wasn't there. I like to taste the coconut,...",1,68,food memorable panang curry balance flavor like taste coconut lemongrass lime lack east asia offer way sweet care owner thai eat anything cook wel...
1,I will never return here again. Ever. I was sitting in my booth waiting for dinner to come out and out scurried a mouse from under my booth and ...,1,87,NOT_return ever sit booth wait dinner come scurry mouse booth dining room immediately get leave inform front desk issue blow nothing say oh ya mou...
2,I wish my experience was great as others. I dined in on a Wednesday night over a week ago and to say the least our meals were horrible. I had the ...,1,166,wish experience great others din wednesday night week ago say least meal horrible jerk chicken entree date seafood coconut entree presentation nic...
3,"Are the rosemary grapefruit scones supposed to taste like a whole box of salt was dumped in the dough? If so, why would you top them with sweet ic...",1,81,rosemary grapefruit scone suppose taste like whole box salt dump dough would top sweet ice like party go mouth NOT_a good party need glass water t...
4,"Our takeout order was half wrong. Food was missing, portion sizes were wrong. If it had been busy, I might be more forgiving, but the place was em...",1,32,takeout order half wrong food miss portion size wrong busy might forgiving place empty 10 10 would NOT_recommend
...,...,...,...,...
9995,Never order Hot Pot here. Too much vegetables and not enough meat. The broth was so plain.,2,17,NOT_order hot pot much vegetable NOT_enough meat broth plain
9996,"I work at BMC and heard about this new place opening. My co-worker and I decided to try it out. First impression walking in, it was bright, clea...",2,253,work bmc hear new place open co worker decide try first impression walking bright clean everyone smile face order pear arugula salad arancini ball...
9997,"Went here for dinner, drinks were very good, food could gave been better. We ordered the Margarita popcorn, it was stale so the waitress took it...",2,150,went dinner drink good food could give well order margarita popcorn stale waitress take back boyfriend order swordfish taco one completely raw tel...
9998,"The food was mediocre. Not horrible not great. I had sausage, brisket and pork ribs. It was all pretty bland and unimpressive. Same goes for the ...",2,64,food mediocre NOT_horrible NOT_great sausage brisket pork rib pretty bland unimpressive go slaw potatoe salad best NOT_great peach cobbler straigh...


In [10]:
# Add additional stop words since we are recreating the document-term matrix

# Create a document-term matrix with only nouns
# Store TF-IDF Vectorizer
tv_noun = TfidfVectorizer(ngram_range = (1,1), max_df = .8, min_df = .01)
# Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
data_tv_noun = tv_noun.fit_transform(df.text_cleaned)
# Create data-frame of Doc-Term Matrix with nouns as column names
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names())
# Set President's Names as Index
data_dtm_noun.index = df.text_cleaned
# Visually inspect Document Term Matrix
data_dtm_noun.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,10,100,11,12,15,18,20,25,30,...,wrong,yeah,year,yell,yelp,yes,yesterday,yet,young,zero
text_cleaned,,,,,,,,,,,,,,,,,,,,,
food memorable panang curry balance flavor like taste coconut lemongrass lime lack east asia offer way sweet care owner thai eat anything cook well place claim serve thai food,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NOT_return ever sit booth wait dinner come scurry mouse booth dining room immediately get leave inform front desk issue blow nothing say oh ya mouse problem NOT_big deal certainly report department health disgust,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wish experience great others din wednesday night week ago say least meal horrible jerk chicken entree date seafood coconut entree presentation nice seafood coconut however date displeased literally bite food couple bite mine mess jerk chicken finish also expect rice peas caribbean restuarant however serve white rice drink ok rum punch drink fruit punch something also patty appetizer ok date say typical frozen patty waitress apologize ask could something disappointed food want try anything else doubt go back,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rosemary grapefruit scone suppose taste like whole box salt dump dough would top sweet ice like party go mouth NOT_a good party need glass water two bite manage swallow vegan friend love twinkie though look great needing palate cleanser first bother try twinkie,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
takeout order half wrong food miss portion size wrong busy might forgiving place empty 10 10 would NOT_recommend,0.0,0.392054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.449125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
#Given an NMF model, feature_names, and number of top words, print topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [12]:
nmf_model = NMF(2)
# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
doc_topic = nmf_model.fit_transform(data_dtm_noun)
# Extract top words from the topic-term matrix 'H' display_topics(nmf_model, tv_noun.get_feature_names(), 5)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [13]:
nmf_model = NMF(15)
doc_topic = nmf_model.fit_transform(data_dtm_noun)
display_topics(nmf_model, tv_noun.get_feature_names(), 5)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,



Topic  0
place, like, really, get, look

Topic  1
good, taste, like, dish, sauce

Topic  2
pizza, crust, cheese, topping, slice

Topic  3
order, take, delivery, get, wrong

Topic  4
food, service, bad, good, slow

Topic  5
table, come, server, waitress, ask

Topic  6
burger, fry, bun, onion, cheese

Topic  7
wait, minute, hour, 30, 15

Topic  8
chicken, rice, fry, wing, sauce

Topic  9
bar, drink, beer, bartender, night

Topic  10
time, last, first, location, second

Topic  11
say, customer, tell, call, manager

Topic  12
sandwich, salad, cheese, bread, lettuce

Topic  13
sushi, roll, fish, rice, salmon

Topic  14
go, back, get, win, use


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
matrice= pd.DataFrame(doc_topic)
matrice

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.022573,0.059246,0.000000,0.000000,0.035336,0.000000,0.000911,0.000000,0.010440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.022893,0.000000,0.026297,0.000480,0.004427,0.000000,0.035595,0.003847,0.000000,0.001985
2,0.000000,0.029418,0.000000,0.003326,0.016264,0.023538,0.000000,0.000000,0.058513,0.025286,0.003248,0.011391,0.000000,0.005930,0.022989
3,0.014022,0.059268,0.000000,0.000000,0.000000,0.014635,0.000185,0.000000,0.000000,0.006729,0.007704,0.003873,0.000000,0.000000,0.019124
4,0.028094,0.005219,0.000000,0.050270,0.017008,0.000000,0.000000,0.017109,0.001041,0.001121,0.000000,0.000000,0.004618,0.001293,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.033370,0.000510,0.003326,0.000000,0.000000,0.000000,0.000000,0.002745,0.000000,0.000000,0.000000,0.004056,0.000000,0.000000
9996,0.034924,0.055417,0.063274,0.008054,0.000000,0.001774,0.001677,0.000000,0.007076,0.000000,0.014835,0.018156,0.057454,0.000000,0.011262
9997,0.000000,0.052196,0.000000,0.034899,0.021410,0.033158,0.000556,0.000000,0.000000,0.017403,0.003645,0.038669,0.000000,0.000000,0.004355
9998,0.000000,0.037690,0.010034,0.000000,0.016528,0.000000,0.000000,0.000000,0.002188,0.000000,0.000000,0.000000,0.043162,0.000000,0.022939


In [15]:
mon_dictionnaire = {0: "good place",1: "good taste", 2: "food", 3: "order", 4: "service", 5: "server", 6: "fast food",7: "time",8: "dishs", 
9: "drink",10: "location",11: "buying",12: "breakfeast", 13: "japanease food", 14: "course"    }  
mon_dictionnaire[0]                 
                    
                    
                    
                    

'good place'

In [16]:
dfr=pd.DataFrame(doc_topic)
dict={}
for i in range(15):
  top3=dfr.nlargest(3, i, keep='all')
  top3Indexes=top3[0].keys().tolist()
  
  dict[i]=df['text'][top3Indexes].tolist()
dfff= pd.DataFrame(dict)
dfff

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"This place used to be a cool, chill place. Now its a bunch of neanderthal bouncers hopped up on steroids acting like the can do whatever they want...",I love Greek food and have eaten at many Greek restaurants in Atlanta and New York. Since I live less than 1.5 miles from this relatively new res...,Ok we tried Mod pizza tonight for 1st time. We are disappointed. 11 inches for a pizza is small barely enough to feed 2 people and definitely not ...,Great prices but I think that they are quantity over quality. I order takeout from here occasionally but orders have been inconsistent lately. CHE...,the food cannot be catogorized as asian food. Worst ever. Bad&slow service and definitly overprice,Unos is actually my favorite restaurant and yet it's come to this--a two star review. This particular Unos I've been to more times than I can cou...,The service was good but they forgot several items off of the burgers we ordered. My guest and I left feeling quite unsatisfied. We sat at the bar...,"The food was decent, not worth the 20 minute wait, and then another HOUR wait for food.\nOverrated, and a waste of time.","This chicken is not all that. It's just lightly fried chicken. The sides are bland as well. Go to Lucy's fried chicken instead, that chicken and s...","Roscoe's is your middle-of-the-road bar located in Montavilla. It's neither upscale nor a dive bar, just a comfy dark pub to enjoy good beer, bar ...",I've been here twice and still think it's OK. The first time with my sister from Virginia. The second time with my significant other and a b...,I called due to a ad on a website provided by Westgate saying that Disney tickets were being sold for $59 called and was told to make a reservatio...,"I'm a sandwich guy. I live for a good sandwich. I love pork also, with a passion. Their porchetta sandwich was gross, basically it was a pork fat ...","I go to an AYCE sushi for the pure goal of satisfying my sushi cravings without breaking my bank. I just want to eat loads of sushi rolls, nigiri,...",i used to like Sidebar. went there often. After my experience today i will not be back. My friend and i went there for lunch and we were treated l...
1,Heard that this is a very popular place in Boston. Went there on a Friday night. The bouncers were rude and racist. Not a ideal place to get inot ...,"First impression: \nWalked into front door and no one was at reception. Problem is, the reception is completely isolated from the rest of the r...","I ordered pizza by the phone call. When I got there the pizza was ready. I gave money, but when I open the box the pizza was burn. They offer me t...",Don't bother pre-ordering online. They don't ask for any order information when you go to pick it up & you end up getting whatever they have - re...,Go for the drinks at the bar. The food is mediocre and the service matches the quality of food.,"If you can get take out, I highly recommend it. food is great, service is not. Torn between 2 stars and 3 because the service totally just kille...","Really rather disappointing, got a burger and fries for takeaway, almost 16 dollars.\n\nburger was meh, fries average. i've tired the burger at th...",Seriously it's worth the wait!? We waited freaking an hour and a half for a table and they had the audacity to give away our table for getting the...,Two recent visits and I have to ask...where's the chicken? Wings were more breaking than chicken ad now the blackened chicken salad had not croton...,We went here on a Saturday night when they had an 80s themed DJ. They were obviously not used to the crowd of people as they had the chef at the ...,"My first time there was great, then my second time was terrible...then I try again and again. But your food quality is so inconsistent that I gave...",Rosie's Bakery is the no good. I'm not an excellent writer so bear with me.\n\nI joined their staff as a trainer several months ago and liked it a...,Stopped here today for lunch after ha

In [17]:
import pickle
with open('modele_file','wb') as file:
  pickle.dump(nmf_model, file)
  file.close()

In [18]:
with open('vectoriseur_file','wb') as file:
  pickle.dump(doc_topic, file)
  file.close()